In [1]:
import os
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import sklearn.metrics as metrics
from math import sqrt
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn import linear_model
from sklearn.linear_model import Lasso
from tpot import TPOTRegressor
#sort
from operator import itemgetter, attrgetter
#gradinentboost
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics

In [2]:
%matplotlib inline
pd.options.display.max_columns = 45
pd.options.display.max_rows = 70

In [3]:

data = pd.read_csv("historical_data1_Q12005.txt",sep="|",header=None)
data.columns = ["CREDIT SCORE","FIRST PAYMENT DATE","FIRST TIME HOMEBUYER FLAG","MATURITY DATE","METROPOLITAN STATISTICAL AREA",
               "MORTGAGE INSURANCE PERCENTAGE","NUMBER OF UNITS","OCCUPANCY STATUS","ORIGINAL COMBINED LOAN-TO-VALU","ORIGINAL DEBT-TO-INCOME (DTI) RATIO",
               "ORIGINAL UPB","ORIGINAL LOAN-TO-VALUE","ORIGINAL INTEREST RATE","CHANNE","PREPAYMENT PENALTY MORTGAGE FLAG",
               "PRODUCT TYPE","PROPERTY STATE","PROPERTY TYPE","POSTAL CODE","LOAN SEQUENCE NUMBER","LOAN PURPOSE",
               "ORIGINAL LOAN TERM","NUMBER OF BORROWERS","SELLER NAME","SERVICER NAME","SUPER CONFORMING FLAG"
                #,"Pre-HARP LOAN SEQUENCE NUMBER"
                ]
data.head()

,CREDIT SCORE,FIRST PAYMENT DATE,FIRST TIME HOMEBUYER FLAG,MATURITY DATE,METROPOLITAN STATISTICAL AREA,MORTGAGE INSURANCE PERCENTAGE,NUMBER OF UNITS,OCCUPANCY STATUS,ORIGINAL COMBINED LOAN-TO-VALU,ORIGINAL DEBT-TO-INCOME (DTI) RATIO,ORIGINAL UPB,ORIGINAL LOAN-TO-VALUE,ORIGINAL INTEREST RATE,CHANNE,PREPAYMENT PENALTY MORTGAGE FLAG,PRODUCT TYPE,PROPERTY STATE,PROPERTY TYPE,POSTAL CODE,LOAN SEQUENCE NUMBER,LOAN PURPOSE,ORIGINAL LOAN TERM,NUMBER OF BORROWERS,SELLER NAME,SERVICER NAME,SUPER CONFORMING FLAG
0,699,200505,N,203504,39300.0,0,1,P,56,42,190000,56,5.625,R,N,FRM,RI,SF,2800.0,F105Q1000001,C,360,2,Other sellers,USBANKNA,NaN
1,691,200504,N,203503,36420.0,25,1,P,90,36,90000,90,5.750,R,N,FRM,OK,SF,73000.0,F105Q1000002,N,360,1,Other sellers,Other servicers,NaN
2,713,200503,N,203502,28740.0,0,1,P,72,45,357000,72,6.000,R,N,FRM,NY,SF,12500.0,F105Q1000003,P,360,2,Other sellers,Other servicers,NaN
3,719,200505,N,203504,NaN,0,1,S,85,47,195000,68,5.750,R,N,FRM,MO,CO,65000.0,F105Q1000004,P,360,2,Other sellers,USBANKNA,NaN
4,656,200503,N,203502,40340.0,0,1,P,68,30,253000,68,5.625,R,N,FRM,MN,SF,55900.0,F105Q1000005,C,360,2,Other sellers,Other servicers,NaN


In [4]:
def preprocessing(data):
    data['POSTAL CODE'].fillna(85200.0,inplace=True)
    data['PREPAYMENT PENALTY MORTGAGE FLAG'].fillna('N',inplace=True)
    data['METROPOLITAN STATISTICAL AREA'].fillna(16974.0,inplace=True)
    data.drop(['SUPER CONFORMING FLAG'],axis=1,inplace=True)
    data['CREDIT SCORE'].replace(to_replace=9999, value=np.nan, inplace=True)
    data['CREDIT SCORE'].fillna((data['CREDIT SCORE'].mean()), inplace=True)
    data['FIRST TIME HOMEBUYER FLAG'].replace(to_replace='9', value=np.nan, inplace=True)
    data['FIRST TIME HOMEBUYER FLAG'].fillna('N', inplace=True)
    data['Maturity_year'] =  [int(str(d)[:4]) for d in data['MATURITY DATE']]
    del data['MATURITY DATE']
    data['MORTGAGE INSURANCE PERCENTAGE'].replace(to_replace=999,value=0,inplace=True)
    data['NUMBER OF UNITS'].replace(to_replace=99,value=1,inplace=True)

    data['ORIGINAL COMBINED LOAN-TO-VALU'].replace(to_replace=999,value=80,inplace=True)

    data['ORIGINAL DEBT-TO-INCOME (DTI) RATIO'].replace(to_replace=999,value=float('nan'),inplace=True)
    data['ORIGINAL DEBT-TO-INCOME (DTI) RATIO'].fillna(data['ORIGINAL DEBT-TO-INCOME (DTI) RATIO'].mean(),inplace=True)

    data['ORIGINAL LOAN-TO-VALUE'].replace(to_replace=999,value=float('nan'),inplace=True)
    data['ORIGINAL LOAN-TO-VALUE'].fillna(data['ORIGINAL LOAN-TO-VALUE'].mean(),inplace=True)

    data.drop('PRODUCT TYPE',axis=1,inplace=True)
    data.drop('LOAN SEQUENCE NUMBER',axis=1,inplace=True)

    data['PROPERTY TYPE'].replace(to_replace=99,value='SF',inplace=True)
    data['LOAN PURPOSE'].replace(to_replace=9,value='C',inplace=True)
    data['NUMBER OF BORROWERS'].replace(to_replace=99,value=2,inplace=True)

    del data['FIRST PAYMENT DATE']

    data.drop('ORIGINAL LOAN-TO-VALUE',axis=1,inplace=True)
    data.drop('Maturity_year',axis=1,inplace=True)

    #dictionary for each column
    cleanup_nums = {"FIRST TIME HOMEBUYER FLAG":{"Y": 1, "N": 0},
                "OCCUPANCY STATUS": {"P": 1, "S": 2, "I": 3 },
                "CHANNE":{"T": 1, "R":2,"C":3,"B":4},
                "PREPAYMENT PENALTY MORTGAGE FLAG":{"Y":1,"N":0},
                "PROPERTY TYPE":{"SF":1,"PU":2,"CO":3,"MH":4,"CP":5,"99":99},
                "LOAN PURPOSE":{"C":1,"P":2,"N":3},
                "SELLER NAME":{"Other sellers": 0, "COUNTRYWIDE": 1, "TAYLOR,BEAN&WHITAKER": 2, "PROVIDENTFUNDINGASSO": 3, "USBANKNA": 4, "FIFTHTHIRDBANK": 5, "ABNAMROMTGEGROUP,INC": 6, "CHASEHOMEFINANCELLC": 7, "NATLCITYMTGECO": 8, "WELLSFARGOBANK,NA": 9, "GMACMTGECORP": 10, "WASHINGTONMUTUALBANK": 11, "FLAGSTARBANK,FSB": 12, "BANKOFAMERICA,NA": 13},
                "PREPAYMENT PENALTY MORTGAGE FLAG":{"Y": 1, "N": 0},
                "PROPERTY STATE":{"RI": 0, "OK": 1, "NY": 2, "MO": 3, "MN": 4, "IL": 5, "KY": 6, "WA": 7, "TX": 8, "FL": 9, "CA": 10, "IN": 11, "NJ": 12, "ID": 13, "TN": 14, "KS": 15, "MI": 16, "IA": 17, "MT": 18, "GA": 19, "OH": 20, "OR": 21, "ME": 22, "CT": 23, "WV": 24, "NH": 25, "VA": 26, "NC": 27, "AZ": 28, "NE": 29, "MD": 30, "MA": 31, "UT": 32, "CO": 33, "ND": 34, "PA": 35, "SC": 36, "DE": 37, "SD": 38, "WI": 39, "AL": 40, "AK": 41, "VT": 42, "LA": 43, "AR": 44, "NM": 45, "HI": 46, "DC": 47, "MS": 48, "NV": 49, "GU": 50, "WY": 51, "PR": 52, "VI": 53},
                "SERVICER NAME":{"USBANKNA": 0, "Other servicers": 1, "PNCMTGESERVICES,INC": 2, "WELLSFARGOBANK,NA": 3, "WASHINGTONMUTUALBANK": 4, "PNCBANK,NATL": 5, "NATLCITYMTGECO": 6, "JPMORGANCHASEBANK,NA": 7, "NATIONSTARMTGELLCDBA": 8, "COUNTRYWIDE": 9, "BANKOFAMERICA,NA": 10, "BACHOMELOANSERVICING": 11, "CITIMORTGAGE,INC": 12, "PROVIDENTFUNDINGASSO": 13, "ABNAMROMTGEGROUP,INC": 14, "FIFTHTHIRDBANK": 15, "GMACMORTGAGE,LLC": 16},
                "OCCUPANCY STATUS":{"P": 1, "S": 2, "I": 3 },
                "CHANNE":{"T": 1, "R":2,"C":3,"B":4}  
               }
    data.replace(cleanup_nums, inplace=True)
    
    #d_seller = {"Other sellers": 0, "COUNTRYWIDE": 1, "TAYLOR,BEAN&WHITAKER": 2, "PROVIDENTFUNDINGASSO": 3, "USBANKNA": 4, "FIFTHTHIRDBANK": 5, "ABNAMROMTGEGROUP,INC": 6, "CHASEHOMEFINANCELLC": 7, "NATLCITYMTGECO": 8, "WELLSFARGOBANK,NA": 9, "GMACMTGECORP": 10, "WASHINGTONMUTUALBANK": 11, "FLAGSTARBANK,FSB": 12, "BANKOFAMERICA,NA": 13}
    #d_service  = {"USBANKNA": 0, "Other servicers": 1, "PNCMTGESERVICES,INC": 2, "WELLSFARGOBANK,NA": 3, "WASHINGTONMUTUALBANK": 4, "PNCBANK,NATL": 5, "NATLCITYMTGECO": 6, "JPMORGANCHASEBANK,NA": 7, "NATIONSTARMTGELLCDBA": 8, "COUNTRYWIDE": 9, "BANKOFAMERICA,NA": 10, "BACHOMELOANSERVICING": 11, "CITIMORTGAGE,INC": 12, "PROVIDENTFUNDINGASSO": 13, "ABNAMROMTGEGROUP,INC": 14, "FIFTHTHIRDBANK": 15, "GMACMORTGAGE,LLC": 16}
    #d_seller.setdefault(data["SELLER NAME"], default=0)
    #d_service.setdefault(data["SERVICER NAME"], default=0)
    
    
    for i in range(len(data['SELLER NAME'])):
        if (not(str(data['SELLER NAME'][i]).isdigit())):
            data['SELLER NAME'][i]=0
    for i in range(len(data['SERVICER NAME'])):
        if (not(str(data['SERVICER NAME'][i]).isdigit())):
            data['SERVICER NAME'][i]=0        
    return data
    

In [5]:
data=preprocessing(data)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351634 entries, 0 to 351633
Data columns (total 20 columns):
CREDIT SCORE                           351634 non-null float64
FIRST TIME HOMEBUYER FLAG              351634 non-null int64
METROPOLITAN STATISTICAL AREA          351634 non-null float64
MORTGAGE INSURANCE PERCENTAGE          351634 non-null int64
NUMBER OF UNITS                        351634 non-null int64
OCCUPANCY STATUS                       351634 non-null int64
ORIGINAL COMBINED LOAN-TO-VALU         351634 non-null int64
ORIGINAL DEBT-TO-INCOME (DTI) RATIO    351634 non-null float64
ORIGINAL UPB                           351634 non-null int64
ORIGINAL INTEREST RATE                 351634 non-null float64
CHANNE                                 351634 non-null int64
PREPAYMENT PENALTY MORTGAGE FLAG       351634 non-null int64
PROPERTY STATE                         351634 non-null int64
PROPERTY TYPE                          351634 non-null int64
POSTAL CODE                

In [8]:
data.head()

,CREDIT SCORE,FIRST TIME HOMEBUYER FLAG,METROPOLITAN STATISTICAL AREA,MORTGAGE INSURANCE PERCENTAGE,NUMBER OF UNITS,OCCUPANCY STATUS,ORIGINAL COMBINED LOAN-TO-VALU,ORIGINAL DEBT-TO-INCOME (DTI) RATIO,ORIGINAL UPB,ORIGINAL INTEREST RATE,CHANNE,PREPAYMENT PENALTY MORTGAGE FLAG,PROPERTY STATE,PROPERTY TYPE,POSTAL CODE,LOAN PURPOSE,ORIGINAL LOAN TERM,NUMBER OF BORROWERS,SELLER NAME,SERVICER NAME
0,699.0,0,39300.0,0,1,1,56,42.0,190000,5.625,2,0,0,1,2800.0,1,360,2,0,0
1,691.0,0,36420.0,25,1,1,90,36.0,90000,5.750,2,0,1,1,73000.0,3,360,1,0,1
2,713.0,0,28740.0,0,1,1,72,45.0,357000,6.000,2,0,2,1,12500.0,2,360,2,0,1
3,719.0,0,16974.0,0,1,2,85,47.0,195000,5.750,2,0,3,3,65000.0,2,360,2,0,0
4,656.0,0,40340.0,0,1,1,68,30.0,253000,5.625,2,0,4,1,55900.0,1,360,2,0,1


In [7]:
val = pd.read_csv("historical_data1_Q22005.txt",sep="|",header=None)
val.columns = ["CREDIT SCORE","FIRST PAYMENT DATE","FIRST TIME HOMEBUYER FLAG","MATURITY DATE","METROPOLITAN STATISTICAL AREA",
               "MORTGAGE INSURANCE PERCENTAGE","NUMBER OF UNITS","OCCUPANCY STATUS","ORIGINAL COMBINED LOAN-TO-VALU","ORIGINAL DEBT-TO-INCOME (DTI) RATIO",
               "ORIGINAL UPB","ORIGINAL LOAN-TO-VALUE","ORIGINAL INTEREST RATE","CHANNE","PREPAYMENT PENALTY MORTGAGE FLAG",
               "PRODUCT TYPE","PROPERTY STATE","PROPERTY TYPE","POSTAL CODE","LOAN SEQUENCE NUMBER","LOAN PURPOSE",
               "ORIGINAL LOAN TERM","NUMBER OF BORROWERS","SELLER NAME","SERVICER NAME","SUPER CONFORMING FLAG"
                #,"Pre-HARP LOAN SEQUENCE NUMBER"
                ]
val.head()

,CREDIT SCORE,FIRST PAYMENT DATE,FIRST TIME HOMEBUYER FLAG,MATURITY DATE,METROPOLITAN STATISTICAL AREA,MORTGAGE INSURANCE PERCENTAGE,NUMBER OF UNITS,OCCUPANCY STATUS,ORIGINAL COMBINED LOAN-TO-VALU,ORIGINAL DEBT-TO-INCOME (DTI) RATIO,ORIGINAL UPB,ORIGINAL LOAN-TO-VALUE,ORIGINAL INTEREST RATE,CHANNE,PREPAYMENT PENALTY MORTGAGE FLAG,PRODUCT TYPE,PROPERTY STATE,PROPERTY TYPE,POSTAL CODE,LOAN SEQUENCE NUMBER,LOAN PURPOSE,ORIGINAL LOAN TERM,NUMBER OF BORROWERS,SELLER NAME,SERVICER NAME,SUPER CONFORMING FLAG
0,715,200508,N,203507,33700.0,0,1,P,58,41,214000,58,5.750,R,N,FRM,CA,SF,95300.0,F105Q2000001,C,360,2,Other sellers,"WELLSFARGOBANK,NA",NaN
1,743,200508,N,203507,NaN,0,1,P,80,36,340000,80,5.875,R,N,FRM,IL,SF,62400.0,F105Q2000002,N,360,2,Other sellers,"PNCMTGESERVICES,INC",NaN
2,772,200508,N,203507,37860.0,0,1,P,80,34,184000,80,5.500,R,N,FRM,FL,PU,32500.0,F105Q2000003,P,360,1,Other sellers,Other servicers,NaN
3,773,200507,N,203506,NaN,0,1,P,64,60,260000,64,5.875,R,N,FRM,MN,SF,56300.0,F105Q2000004,N,360,2,Other sellers,USBANKNA,NaN
4,647,200508,N,203507,NaN,30,1,P,91,28,170000,91,6.000,R,N,FRM,MI,SF,49200.0,F105Q2000005,N,360,1,Other sellers,Other servicers,NaN


In [8]:
val=preprocessing(val)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405679 entries, 0 to 405678
Data columns (total 20 columns):
CREDIT SCORE                           405679 non-null float64
FIRST TIME HOMEBUYER FLAG              405679 non-null int64
METROPOLITAN STATISTICAL AREA          405679 non-null float64
MORTGAGE INSURANCE PERCENTAGE          405679 non-null int64
NUMBER OF UNITS                        405679 non-null int64
OCCUPANCY STATUS                       405679 non-null int64
ORIGINAL COMBINED LOAN-TO-VALU         405679 non-null int64
ORIGINAL DEBT-TO-INCOME (DTI) RATIO    405679 non-null float64
ORIGINAL UPB                           405679 non-null int64
ORIGINAL INTEREST RATE                 405679 non-null float64
CHANNE                                 405679 non-null int64
PREPAYMENT PENALTY MORTGAGE FLAG       405679 non-null int64
PROPERTY STATE                         405679 non-null int64
PROPERTY TYPE                          405679 non-null int64
POSTAL CODE                

In [10]:
val["SELLER NAME"].value_counts()

9     87433
0     64446
11    42319
7     42063
1     38506
6     34367
8     22091
4     18668
10    15275
13    13751
2     12114
5      5442
3      4816
12     4388
Name: SELLER NAME, dtype: int64

In [10]:
(train_x,train_y)=(data.drop('ORIGINAL INTEREST RATE',axis=1),data['ORIGINAL INTEREST RATE'])

In [11]:
(test_x,test_y)=(val.drop('ORIGINAL INTEREST RATE',axis=1),val['ORIGINAL INTEREST RATE'])

In [33]:
val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405679 entries, 0 to 405678
Data columns (total 20 columns):
CREDIT SCORE                           405679 non-null float64
FIRST TIME HOMEBUYER FLAG              405679 non-null int64
METROPOLITAN STATISTICAL AREA          405679 non-null float64
MORTGAGE INSURANCE PERCENTAGE          405679 non-null int64
NUMBER OF UNITS                        405679 non-null int64
OCCUPANCY STATUS                       405679 non-null int64
ORIGINAL COMBINED LOAN-TO-VALU         405679 non-null int64
ORIGINAL DEBT-TO-INCOME (DTI) RATIO    405679 non-null float64
ORIGINAL UPB                           405679 non-null int64
ORIGINAL INTEREST RATE                 405679 non-null float64
CHANNE                                 405679 non-null int64
PREPAYMENT PENALTY MORTGAGE FLAG       405679 non-null int64
PROPERTY STATE                         405679 non-null int64
PROPERTY TYPE                          405679 non-null int64
POSTAL CODE                

In [18]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [15]:
#Linear Regression
lin_reg = LinearRegression()
lin_reg.fit(train_x,train_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [16]:
pred_y = lin_reg.predict(test_x)
print("Score for train: "+str(lin_reg.score(train_x,train_y)))
print("Score: "+str(lin_reg.score(test_x,test_y)))
print("RMS: "+str(sqrt(metrics.mean_squared_error(test_y,pred_y))))
print("MAPE: "+str(mean_absolute_percentage_error(test_y,pred_y)))
print("R2: "+str(metrics.r2_score(test_y,pred_y)))
print("MAE: "+str(metrics.mean_absolute_error(test_y,pred_y)))

Score for train: 0.3859683041751738
Score: 0.1564093827436709
RMS: 0.32200672051113916
MAPE: 4.206381090626703
R2: 0.1564093827436709
MAE: 0.24650081474948188


In [17]:
#Lasso
lasso= linear_model.Lasso()
lasso.fit(train_x, train_y)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [18]:
pred_y = lasso.predict(test_x)
print("Score for train: "+str(lasso.score(train_x,train_y)))
print("Score: "+str(lasso.score(test_x,test_y)))
print("RMS: "+str(sqrt(metrics.mean_squared_error(test_y,pred_y))))
print("MAPE: "+str(mean_absolute_percentage_error(test_y,pred_y)))
print("R2: "+str(metrics.r2_score(test_y,pred_y)))
print("MAE: "+str(metrics.mean_absolute_error(test_y,pred_y)))

Score for train: 0.32599145587251577
Score: 0.10417716818060818
RMS: 0.3318257838937045
MAPE: 4.271912138617848
R2: 0.10417716818060818
MAE: 0.2508994814134796


In [19]:
#decision tree
dt_reg = DecisionTreeRegressor(max_depth=9)
min=0
for i in range(8,18):
    rfe = RFE(dt_reg,i)
    rfe = rfe.fit(train_x,train_y)
    pred_y = rfe.predict(test_x)
    score = rfe.score(test_x,test_y)
    print("Score for "+str(i)+" feature: "+str(score))
    if (score>min) :
        min = score
        best_rfe = rfe
pred_y = best_rfe.predict(test_x)
print("Score for train: "+str(best_rfe.score(train_x,train_y)))
print("Score: "+str(best_rfe.score(test_x,test_y)))
print("RMS: "+str(sqrt(metrics.mean_squared_error(test_y,pred_y))))
print("MAPE: "+str(mean_absolute_percentage_error(test_y,pred_y)))
print("R2: "+str(metrics.r2_score(test_y,pred_y)))
print("MAE: "+str(metrics.mean_absolute_error(test_y,pred_y)))

Score for 8 feature: 0.18565568524220677
Score for 9 feature: 0.18387464073922297
Score for 10 feature: 0.18554127889669514
Score for 11 feature: 0.18507043971333326
Score for 12 feature: 0.18587418131539699
Score for 13 feature: 0.18494955642249655
Score for 14 feature: 0.1844081202577109
Score for 15 feature: 0.1842784223755105
Score for 16 feature: 0.18412198454800133
Score for 17 feature: 0.18407146733639432
Score for train: 0.4472722598159683
Score: 0.18587418131539699
RMS: 0.3163332398987283
MAPE: 4.136577041313124
R2: 0.185874181315397
MAE: 0.24220293306395663


In [20]:
test_x=test_x.iloc[:,pd.Series(best_rfe.support_).values]

In [ ]:
#random forest
rf_reg = RandomForestRegressor(max_depth=20,n_estimators=50,verbose=2)
rf_reg.fit(train_x,train_y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 50


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.0s remaining:    0.0s


building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50


In [69]:
pred_y = rf_reg.predict(test_x)
print("Score for train: "+str(rf_reg.score(train_x,train_y)))
print("Score: "+str(rf_reg.score(test_x,test_y)))
print("RMS: "+str(sqrt(metrics.mean_squared_error(test_y,pred_y))))
print("MAPE: "+str(mean_absolute_percentage_error(test_y,pred_y)))
print("R2: "+str(metrics.r2_score(test_y,pred_y)))
print("MAE: "+str(metrics.mean_absolute_error(test_y,pred_y)))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    2.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    2.3s finished


Score for train: 0.7350050354334727


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    2.6s finished


Score: 0.25197806979291015
RMS: 0.3032188740519677
MAPE: 3.985304929866683
R2: 0.25197806979291015
MAE: 0.23332288640279422


In [30]:
#mlp
transformer = Normalizer().fit(train_x)
mlp_train_x = pd.DataFrame(transformer.transform(train_x))
mlp_train_x.columns = train_x.columns

mlp_test_x = Normalizer().fit(test_x)
mlp_test_x = pd.DataFrame(transformer.transform(test_x))
mlp_test_x.columns = test_x.columns

In [36]:
mlp_reg = MLPRegressor(hidden_layer_sizes=(20,10),learning_rate='adaptive',solver='adam',random_state=3,learning_rate_init=0.001,max_iter=400,verbose=True)
mlp_reg.fit(mlp_train_x,train_y)

Iteration 1, loss = 0.78170768
Iteration 2, loss = 0.07030542
Iteration 3, loss = 0.06856018
Iteration 4, loss = 0.06600670
Iteration 5, loss = 0.06313847
Iteration 6, loss = 0.06016092
Iteration 7, loss = 0.05680422
Iteration 8, loss = 0.05292272
Iteration 9, loss = 0.04990944
Iteration 10, loss = 0.04832167
Iteration 11, loss = 0.04774766
Iteration 12, loss = 0.04719782
Iteration 13, loss = 0.04689246
Iteration 14, loss = 0.04658359
Iteration 15, loss = 0.04632161
Iteration 16, loss = 0.04599998
Iteration 17, loss = 0.04580417
Iteration 18, loss = 0.04553888
Iteration 19, loss = 0.04534115
Iteration 20, loss = 0.04520735
Iteration 21, loss = 0.04502538
Iteration 22, loss = 0.04493507
Iteration 23, loss = 0.04480682
Iteration 24, loss = 0.04478218
Iteration 25, loss = 0.04470437
Iteration 26, loss = 0.04464144
Iteration 27, loss = 0.04470527
Iteration 28, loss = 0.04467441
Iteration 29, loss = 0.04462062
Iteration 30, loss = 0.04455876
Iteration 31, loss = 0.04458090
Iteration 32, los

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(20, 10), learning_rate='adaptive',
       learning_rate_init=0.001, max_iter=400, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=3, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=True, warm_start=False)

In [ ]:
pred_y = mlp_reg.predict(mlp_test_x)
print("Score for train: "+str(mlp_reg.score(mlp_train_x,train_y)))
print("Score: "+str(mlp_reg.score(mlp_test_x,test_y)))
print("RMS: "+str(sqrt(metrics.mean_squared_error(test_y,pred_y))))
print("MAPE: "+str(mean_absolute_percentage_error(test_y,pred_y)))
print("R2: "+str(metrics.r2_score(test_y,pred_y)))
print("MAE: "+str(metrics.mean_absolute_error(test_y,pred_y)))

In [91]:
#gradient boost
gbm_reg = GradientBoostingRegressor(verbose=2,learning_rate=0.8)
gbm_reg.fit(train_x,train_y)

      Iter       Train Loss   Remaining Time 
         1           0.0908           41.81s
         2           0.0841           44.72s
         3           0.0804           43.03s
         4           0.0787           42.65s
         5           0.0777           42.46s
         6           0.0765           42.00s
         7           0.0757           41.71s
         8           0.0752           42.19s
         9           0.0747           41.70s
        10           0.0743           41.39s
        11           0.0741           40.86s
        12           0.0736           40.77s
        13           0.0733           40.50s
        14           0.0730           40.45s
        15           0.0726           39.93s
        16           0.0723           39.11s
        17           0.0721           38.60s
        18           0.0719           38.39s
        19           0.0718           37.75s
        20           0.0716           37.19s
        21           0.0714           36.83s
        2

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.8, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=2, warm_start=False)

In [92]:
pred_y = gbm_reg.predict(test_x)
print("Score for train: "+str(gbm_reg.score(train_x,train_y)))
print("Score: "+str(gbm_reg.score(test_x,test_y)))
print("RMS: "+str(sqrt(metrics.mean_squared_error(test_y,pred_y))))
print("MAPE: "+str(mean_absolute_percentage_error(test_y,pred_y)))
print("R2: "+str(metrics.r2_score(test_y,pred_y)))
print("MAE: "+str(metrics.mean_absolute_error(test_y,pred_y)))

Score for train: 0.5112824043609143
Score: 0.2447740044327562
RMS: 0.30467549869213406
MAPE: 4.001492992471372
R2: 0.2447740044327562
MAE: 0.2343171841629704


In [13]:
#Tpot
tpot = TPOTRegressor(generations=3,scoring='r2',population_size=10,verbosity=2)
tpot.fit(train_x,train_y)

Generation 1 - Current best internal CV score: 0.3313925168040158
Generation 2 - Current best internal CV score: 0.3313925168040158
Generation 3 - Current best internal CV score: 0.3313925168040158

Best pipeline: ExtraTreesRegressor(DecisionTreeRegressor(input_matrix, max_depth=7, min_samples_leaf=1, min_samples_split=7), bootstrap=True, max_features=0.1, min_samples_leaf=1, min_samples_split=2, n_estimators=100)


TPOTRegressor(config_dict=None, crossover_rate=0.1, cv=5,
       disable_update_check=False, early_stop=None, generations=3,
       max_eval_time_mins=5, max_time_mins=None, memory=None,
       mutation_rate=0.9, n_jobs=1, offspring_size=None,
       periodic_checkpoint_folder=None, population_size=10,
       random_state=None, scoring='r2', subsample=1.0, use_dask=False,
       verbosity=2, warm_start=False)

In [26]:

tpot = TPOTRegressor(generations=4,scoring='r2',population_size=8,verbosity=2)
tpot.fit(train_x,train_y)

Generation 1 - Current best internal CV score: 0.21717019315911953
Generation 2 - Current best internal CV score: 0.21717019315911953
Generation 3 - Current best internal CV score: 0.22487331221498633
Generation 4 - Current best internal CV score: 0.3461787857248448

Best pipeline: RandomForestRegressor(input_matrix, bootstrap=True, max_features=0.25, min_samples_leaf=8, min_samples_split=7, n_estimators=100)


TPOTRegressor(config_dict=None, crossover_rate=0.1, cv=5,
       disable_update_check=False, early_stop=None, generations=4,
       max_eval_time_mins=5, max_time_mins=None, memory=None,
       mutation_rate=0.9, n_jobs=1, offspring_size=None,
       periodic_checkpoint_folder=None, population_size=8,
       random_state=None, scoring='r2', subsample=1.0, use_dask=False,
       verbosity=2, warm_start=False)

In [ ]:
tpot.score(test_x,test_y)

In [29]:
tpot.export("assignment3.py")

True